In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr').getOrCreate()

In [3]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/ContainsNull.csv,ContainsNull.csv,61
dbfs:/FileStore/tables/Ecommerce_Customers.csv,Ecommerce_Customers.csv,86871
dbfs:/FileStore/tables/Spark_Essentials-5d27c.dbc,Spark_Essentials-5d27c.dbc,1414841
dbfs:/FileStore/tables/appl_stock.csv,appl_stock.csv,143130
dbfs:/FileStore/tables/cogsley_clients.csv,cogsley_clients.csv,384219
dbfs:/FileStore/tables/cogsley_sales.csv,cogsley_sales.csv,2176442
dbfs:/FileStore/tables/people.json,people.json,73
dbfs:/FileStore/tables/sales_info.csv,sales_info.csv,196
dbfs:/FileStore/tables/sample_linear_regression_data.txt,sample_linear_regression_data.txt,119069
dbfs:/FileStore/tables/state_info.csv,state_info.csv,2778


In [4]:
path = '/FileStore/tables/Ecommerce_Customers.csv'
df = spark.read.csv(path,inferSchema= True,header=True)

In [5]:
df.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
 Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
mstephenson@ferna...|835 Frank TunnelW...| Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616| 4.0826206329529615| 587.9510539684005|
 hduke@hotmail.com|4547 Archer Commo...| DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744| 2.66403418213262| 392.2049334443264|
 pallen@yahoo.com|24645 Valerie Uni...| Bisque|33.000914755642675|11.330278057777512|37.110597442120856| 4.104543202376424| 487.54750486747207|
riverarebecca@gma...|1414 David Throug...| SaddleBrown| 34.30555662975554|13.717513665142507| 36.72128267790313| 3.120178782748092| 581.8523440352177|
mstephens@davidso...|14023 Rodriguez P...|MediumAquaMarine| 33.33067252364639|12.795188551078114| 37.53665330059473| 4.446308318351434| 599.4060920457634|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
only showing top 5 rows

In [6]:
df.describe().show()

+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
summary| Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
 count| 500| 500| 500| 500| 500| 500| 500| 500|
 mean| null| null| null| 33.05319351819619|12.052487937166134| 37.06044542094859| 3.533461555915055| 499.3140382585909|
 stddev| null| null| null|0.9925631110845354|0.9942156084725424|1.0104889067564033| 0.9992775024112585| 79.3147815497068|
 min|aaron04@yahoo.com|0001 Mack MillNor...| AliceBlue|29.532428967057943| 8.508152176032603| 33.91384724758464| 0.2699010899842742| 256.67058229005585|
 max|zscott@wright.com|Unit 7502 Box 834...|YellowGreen| 36.13966248879052|15.126994288792467|40.005181638101895| 6.922689335035808| 765.5184619388373|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+

In [7]:
df.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
df.columns

Out[ 10 ]: 
['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
# taking the interesting variables and transforming into one feature vector named features.This featureS column is the independent variable
#spark expect this format
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],
                           outputCol='features')

In [11]:
output = assembler.transform(df)

In [12]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [13]:
output.show(2)

+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
 Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent| features|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
mstephenson@ferna...|835 Frank TunnelW...| Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616| 4.0826206329529615| 587.9510539684005|[34.4972677251122...|
 hduke@hotmail.com|4547 Archer Commo...|DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744| 2.66403418213262| 392.2049334443264|[31.9262720263601...|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
only showing top 2 rows

In [14]:
final_data = output.select(['features','Yearly Amount Spent'])

In [15]:
final_data.show(5) 
# here features is the vector of independent varibles  and yearly amount is the depndent variable

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows

In [16]:
train_data,test_data= final_data.randomSplit([0.7,0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 351|
 mean| 498.95718543181215|
 stddev| 79.58174877713823|
 min| 256.67058229005585|
 max| 744.2218671047146|
+-------+-------------------+

In [18]:
from pyspark.ml.regression import LinearRegression

In [19]:
lr = LinearRegression(featuresCol='features',labelCol='Yearly Amount Spent',predictionCol='prediction')

In [20]:
lr_model = lr.fit(train_data)

In [21]:
model_summary = lr_model.summary

In [22]:
model_summary.pValues

Out[ 31 ]: [0.0, 0.0, 0.03771631117619867, 0.0, 0.0]

In [23]:
model_summary.rootMeanSquaredError

Out[ 32 ]: 9.724928338630889

In [24]:
model_summary.r2

Out[ 33 ]: 0.9850243757439365

In [25]:
model_summary.residuals.show()

+-------------------+
 residuals|
+-------------------+
 8.900372520593692|
 -18.65307821338081|
 -4.629434673915455|
 -4.797454536989505|
 6.569749638604435|
 2.3343409473008023|
 -6.464926578752113|
 -13.57847629244651|
-21.432975393189167|
 8.923880048141655|
 22.215413132413914|
 16.867286093408495|
 3.37236996873969|
 2.813347768223025|
-1.0653358458837374|
 -4.903993911450243|
 -7.904582378966779|
 -6.372266048205461|
 1.3704765032410364|
-14.747910747719743|
+-------------------+
only showing top 20 rows

In [26]:
test_result = lr_model.evaluate(test_data)

In [27]:
test_result.r2

Out[ 41 ]: 0.9821749211439911

In [28]:
test_result.rootMeanSquaredError

Out[ 42 ]: 10.504308837170319

In [29]:
final_data.describe().show()
# here the mean of  y is 499.31 and sd is 79.31 while the root mean sqaure error of y on test data is 10.5. this means the model is fitted well

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [30]:
unlabeled_data = test_data.select('features')

In [31]:
pred = lr_model.transform(unlabeled_data)

In [32]:
pred.show(5)

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[30.3931845423455...|332.71959683066416|
[30.4925366965402...|288.67345170902627|
[30.5743636841713...| 442.963331163955|
[30.7377203726281...| 451.7720645465679|
[31.1280900496166...| 565.5844992285201|
+--------------------+------------------+
only showing top 5 rows

In [33]:
pred.show(5)

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[30.3931845423455...|332.71959683066416|
[30.4925366965402...|288.67345170902627|
[30.5743636841713...| 442.963331163955|
[30.7377203726281...| 451.7720645465679|
[31.1280900496166...| 565.5844992285201|
+--------------------+------------------+
only showing top 5 rows